<a href="https://colab.research.google.com/github/jia020/myJupyterNotebooks/blob/master/NVCL_NMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NMM(National Mineral Map) For NVCL datasets.

In [ ]:
pip install nvcl_kit

In [2]:
#import all libs for olivine
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import BeautifyIcon
from branca.element import Figure

pd.options.mode.chained_assignment = None
pd.set_option('display.width', 400)

In [3]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')
scalar ='Fayalite'
path=f'/content/drive/My Drive/work/{scalar}/'
analyticalFileList = ['passIds.csv','failIds.csv','errorIds.csv','passIds0.csv','failIds0.csv','errorIds0.csv']


Mounted at /content/drive


In [ ]:
#build xy-state.csv for each state, NT take really long time.
#If you aleady  have file xy-all-5645.csv, you could skip this code cell.
from nvcl_kit.reader import NVCLReader
from nvcl_kit.param_builder import param_builder
xy_dict = {}
statesList = ['nsw', 'tas', 'vic', 'qld', 'nt', 'sa', 'wa', 'csiro']
for state in statesList:
  param = param_builder(state, bbox = { "west": 110, "south": -46, "east": 155, "north": -6 })#bbox for whole Australia
  if not param:
    print(f"Cannot build parameters for {state}")
    break
  reader = NVCLReader(param)
  if not reader.wfs:
    print(f"No WFS for {state}")
    break
  bh_list = reader.get_boreholes_list()
  dfW = pd.DataFrame(bh_list, columns=['href', 'x','y','name','state']) #.sample(n=500)
  dfW['state'] = state
  num = dfW.shape[0]
  dfW.to_csv(f'{path}xy-{state}.csv', sep=',', encoding='utf-8')


In [ ]:
#buildup xy-all-5645.csv
#If you aleady  have file xy-all-5645.csv, you could skip this code cell.

statesList = ['nsw', 'tas', 'vic', 'qld', 'nt', 'sa', 'wa', 'csiro']

df0 = pd.DataFrame(columns=['href', 'x','y','name','state'])
for state in statesList:
  df1 = pd.read_csv(f'{path}xy-{state}.csv')
  df0 = pd.merge(df0[['href', 'x','y','name','state']], df1[['href', 'x','y','name','state']],  how = "outer")
  num = df0.shape[0]
  print(f'{num}')
dfW.to_csv(f'{path}xy-all-5645.csv', sep=',', encoding='utf-8')


In [ ]:
#buildup olivine.csv (merge from error.csv + fail.csv + pass.csv)
#fileList = ['passIds.csv','failIds.csv','errorIds.csv','passIds0.csv','failIds0.csv','errorIds0.csv']
li = []
for file in analyticalFileList:
  try:
    df5 = pd.read_csv(f'{path}{file}',names = ['BoreholeURI','scalar'], index_col=False, sep=',')
    li.append(df5)
  except pd.errors.EmptyDataError:
    print(f'{path}{file} is empty')

df5 = pd.concat(li, axis=0, ignore_index=True)
dframe = df5[['BoreholeURI','scalar']]
dframe.to_csv(f'{path}scalar.csv',index=False,sep=',', encoding='utf-8')

In [ ]:
#buildup xy-olivine.csv
df = pd.read_csv(f'{path}scalar.csv')
xy_dict = pd.read_csv(f'{path}xy-all-5645.csv').set_index('href').to_dict(orient='index')
print(f'xy_dict size is {len(xy_dict)}')

dfW = pd.DataFrame(columns=['scalar', 'State',	'BoreholeName',	'BoreholeURI',	'Latitude',	'Longitude'])

t=0
p=0
f=0
e=0
scalarTest = 'fail'
for row in df.itertuples():
  if(row.scalar.find('Hit:')>=0):
    scalarTest='pass'
    p+=1
  elif(row.scalar.find('Miss:')>=0):
    scalarTest='fail'
    f+=1
  else:
    scalarTest='error'
    e+=1
  dfW.loc[t] = [scalarTest,xy_dict[row.BoreholeURI]['state'],xy_dict[row.BoreholeURI]['name'],row.BoreholeURI,xy_dict[row.BoreholeURI]['y'],xy_dict[row.BoreholeURI]['x']]
  t+=1
#
print(f'p:{p},f:{f},e:{e},t:{t}')
dfW.to_csv(f'{path}xy-scalar.csv', index=False, sep=',', encoding='utf-8')



xy_dict size is 5645
p:12,f:1272,e:4213,t:5497


In [ ]:
#show scalar on map
dfScalar=pd.read_csv(f'{path}xy-scalar.csv')
#[['scalar', 'State',	'BoreholeName',	'BoreholeURI',	'Latitude',	'Longitude']]
cx = dfScalar['Longitude'].mean()
cy = dfScalar['Latitude'].mean()
fig2=Figure(width=1024,height=768)
m2=folium.Map(location=[cy,cx],zoom_start=4)
fig2.add_child(m2)
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Maps',overlay = False,control = True),
    'Google Satellite': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Satellite',overlay = False,control = True),
    'Google Terrain': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Terrain',overlay = False,control = True),
    'Google Satellite Hybrid': folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',attr = 'Google',name = 'Google Satellite',overlay = False,control = True),
    'Esri Satellite': folium.TileLayer(tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',attr = 'Esri',name = 'Esri Satellite',overlay = False,control = True)
}
basemaps['Google Maps'].add_to(m2)
basemaps['Google Satellite Hybrid'].add_to(m2)
basemaps['Google Satellite'].add_to(m2)
basemaps['Google Terrain'].add_to(m2)
basemaps['Esri Satellite'].add_to(m2)
folium.LayerControl().add_to(m2)
#add circle-dot for scalar boreholes:
#blue is pass
#red is fail
#yellow is error
cc=0
for row in dfScalar.itertuples():
  if row.scalar == 'pass' :
    folium.Marker(location=[row.Latitude,row.Longitude],popup=row.BoreholeName,icon=BeautifyIcon(icon_shape='circle-dot', border_color='blue', border_width=3,)).add_to(m2)
  elif row.scalar == 'fail':
    folium.Marker(location=[row.Latitude,row.Longitude],popup=row.BoreholeName,icon=BeautifyIcon(icon_shape='circle-dot', border_color='red', border_width=3,)).add_to(m2)
    cc = cc
  else:
    folium.Marker(location=[row.Latitude,row.Longitude],popup=row.BoreholeName,icon=BeautifyIcon(icon_shape='circle-dot', border_color='yellow', border_width=3,)).add_to(m2)
    cc = cc
  cc+=1
m2.save(f'{path}scalar-au-map.html')
m2

In [4]:
from nvcl_kit.reader import NVCLReader
from nvcl_kit.param_builder import param_builder
xy_dict = {}
statesList = ['tas'] #['nsw', 'tas', 'vic', 'qld', 'nt', 'sa', 'wa', 'csiro']
for state in statesList:
  param = param_builder(state, bbox = { "west": 110, "south": -46, "east": 155, "north": -6 })#bbox for whole Australia
  if not param:
    print(f"Cannot build parameters for {state}")
    break
  reader = NVCLReader(param)
  if not reader.wfs:
    print(f"No WFS for {state}")
    break

In [ ]:
#Download scalar spec data as csv for borehole nvcl_id
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import io
import array
nvcl_id = '6200'#'10350'

#pass,tas,ML048,http://www.mrt.tas.gov.au/resource/feature/mrt/borehole/10350,-41.69823623,145.32070134
#pass,tas,Forest-1,http://www.mrt.tas.gov.au/resource/feature/mrt/borehole/6200,-40.8151763,145.25523887

#SWIR group
#algIDS=['75','89','109','41','33','2','13','49']

#SWIR Mineral
#algIDS=['74','88','108','40','32','1','12','48']

#TIR group
#algIDS=['69','103','119','127','135','63','57','143']

#TIR Mineral
algIDS=['68','102','118','126','134','62','56','142']

# Log name has 2 parts:
# 1. Min1,2,3 = 1st, 2nd, 3rd most common mineral
#    OR Grp1,2,3 = 1st, 2nd, 3rd most common group of minerals
# 2. uTSAV = visible light, uTSAS = shortwave IR, uTSAT = thermal IR
#
# These combine to give us a class name such as 'Grp1 uTSAS'
#
# Here we extract data for log type '1' and log name 'Grp1 uTSAS'
print(f'start download spec for BH:{nvcl_id}')
logs_data_list = reader.get_logs_data(nvcl_id)
scallist=[]
for ld in logs_data_list:
  if ld.log_type == '1' and ld.algorithm_id in algIDS and ld.is_public=='true':
    scallist.append(ld.log_id)

bh_data = reader.get_scalar_data(scallist)
buffer = io.StringIO(bh_data.decode('utf-8'))
df = pd.read_csv(filepath_or_buffer = buffer)
dfS=df.loc[ (df.iloc[:,2] == scalar) | (df.iloc[:,3] == scalar) | (df.iloc[:,4] == scalar) ]

####################################################################
speclogs = reader.get_spectrallog_data(nvcl_id)
dfSS = dfS
for index, row in dfS.iterrows():
  for speclog in speclogs:
    if speclog.sample_count > 0:
      wvName=f'WV-{speclog.wavelengths[0]}-{speclog.wavelengths[1]-speclog.wavelengths[0]}-{speclog.wavelengths[-1]}'.replace('.','_')
      if wvName not in dfSS.columns:
        dfSS[wvName] = ''
      spec_data=reader.get_spectrallog_datasets(speclog.log_id,start_sample_no=index,end_sample_no=index)
      U = array.array("f")
      U.frombytes(spec_data)
      spec_dataS=",".join(map(str, U))
      dfSS.loc[index,wvName] = spec_dataS
#save the raw spectrum into file
dfSS.to_csv(f'{path}{scalar}-spec-{nvcl_id}.csv', index=False, sep=',', encoding='utf-8')
print(f'total specs:{len(dfSS)}')
print(dfSS.head(5))


In [ ]:
#Plot the spectrum chart
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
#load spectrum from file
dfSS = pd.read_csv(f'{path}{scalar}-spec-{nvcl_id}.csv')
#buildup wvLengths array from column name
wv1 = dfSS.columns[-1]
wv11 = wv1.split('-')
wv1Start = float(wv11[1].replace('_','.'))
wv1Step= float(wv11[2].replace('_','.'))
wv1End = float(wv11[3].replace('_','.'))
wv1Lengths = np.arange(wv1Start, wv1End+wv1Step,wv1Step,dtype=float)

wv2 = dfSS.columns[-2]
wv22 = wv2.split('-')
wv2Start = float(wv22[1].replace('_','.'))
wv2Step= float(wv22[2].replace('_','.'))
wv2End = float(wv22[3].replace('_','.'))
wv2Lengths = np.arange(wv2Start, wv2End+wv2Step,wv2Step,dtype=float)
#init a empty dataframe
cnames=['wvLengths']
df1 = pd.DataFrame(columns=cnames)
df1['wvLengths'] = wv1Lengths
df2= pd.DataFrame(columns=cnames)
df2['wvLengths'] = wv2Lengths
#fillup the spec column into dataframe
for index, row in dfSS.iterrows():
  specName = f'spec{index}'
  spec1 = np.array(row[wv1].split(','),dtype=np.float32)
  spec2 = np.array(row[wv2].split(','),dtype=np.float32)
  df1[specName] = spec1
  df2[specName] = spec2

#plot all specs
fig,ax = plt.subplots()
for col in df1.columns:
  if col == 'wvLengths':
    continue
  ax.plot(df1['wvLengths'],df1[col],label=col)

fig1,ax1 = plt.subplots()
for col in df2.columns:
  if col == 'wvLengths':
    continue
  ax1.plot(df2['wvLengths'],df2[col],label=col)

